In [1]:
# Step-2:
#### 1. Image Contrast
#### 2. Data Split (Train(89), Validation(20))
#### 3. Transfer learning

In [1]:
import numpy as np
import tensorflow
import keras

In [4]:
trainingDataPath = 'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/TrainingDataset'

In [6]:
trainData = keras.utils.image_dataset_from_directory(
                                    trainingDataPath,
                                    seed=21,
                                    image_size=(128, 128),
                                    batch_size=32
)

Found 9625 files belonging to 2 classes.


In [9]:
trainData.class_names

['0', '1']

In [10]:
dataGenerator = trainData.as_numpy_iterator()